In [4]:
import tensorflow as tf 
import numpy as np 
import random 


seed_value = 42 
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
random.seed(seed_value)

In [9]:

from tensorflow import keras

# Define your target image size
target_size = (224, 224) # A common size, but adjust based on your model/needs

# 1. Load the Training Dataset
# Point directly to the 'train' subdirectory.
# 'labels='inferred'' will use the subdirectory names ('rural', 'urban') as class labels.
# 'label_mode='categorical'' will one-hot encode these labels (e.g., [1,0] for rural, [0,1] for urban).
# No validation_split needed here as you have a separate 'val' directory.
train_ds = tf.keras.utils.image_dataset_from_directory(
    'rural_and_urban_photos/train',
    labels='inferred',
    label_mode='categorical',
    image_size=target_size,
    interpolation='nearest', # Or 'bilinear' for smoother resizing
    batch_size=32,
    shuffle=True, # Always shuffle training data
    seed=123 # For reproducibility of shuffling
)

# 2. Load the Validation Dataset
# Point directly to the 'val' subdirectory.
# Crucially, DO NOT use validation_split or subset here,
# as the entire 'val' directory is intended for validation.
val_ds = tf.keras.utils.image_dataset_from_directory(
    'rural_and_urban_photos/val',
    labels='inferred',
    label_mode='categorical',
    image_size=target_size,
    interpolation='nearest', # Consistent with training set
    batch_size=32,
    shuffle=False, # No need to shuffle validation data (unless you explicitly want to display random samples)
    # No 'seed' needed if shuffle is False and no split
)

# OPTIONAL: Get class names (useful for understanding your labels)
class_names = train_ds.class_names
print(f"Class names: {class_names}") # Will likely output ['rural', 'urban']
# OPTIONAL: Prefetching and Caching for performance
# These make sure your data pipeline is efficient
train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

# Now, your `train_ds` and `val_ds` are ready to be used in your Keras model's `fit()` method:
# model.fit(train_ds, validation_data=val_ds, epochs=...)

Found 72 files belonging to 2 classes.
Found 20 files belonging to 2 classes.
Class names: ['rural', 'urban']


In [14]:


from  keras.models import Sequential 
from  keras.layers import Input,Conv2D,MaxPool2D,Flatten,Dense,Dropout 

# It's good practice to define your input shape based on target_size from your data loading
# Make sure this matches the image_size you used in image_dataset_from_directory
IMAGE_HEIGHT = 224
IMAGE_WIDTH = 224
IMAGE_CHANNELS = 3 # RGB images have 3 channels

model = Sequential()

# Input Layer: Specifies the expected shape of the input images
# (height, width, channels)
model.add(Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS)))

# First Convolutional Block
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2))) # <--- Corrected: Added closing parenthesis
model.add(Dropout(0.25))

# Second Convolutional Block
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu')) # Increased filters for deeper layers
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu')) # Consistent filter increase
model.add(MaxPool2D(pool_size=(2, 2))) # <--- Corrected: Added closing parenthesis
model.add(Dropout(0.25))

# Flatten Layer: Converts the 2D feature maps into a 1D vector
# This is necessary before feeding into Dense (fully connected) layers
model.add(Flatten())

# Dense (Fully Connected) Layers
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.5))

# Output Layer:
# - `Dense(2, ...)`: 2 neurons because you have 2 classes (rural, urban).
# - `activation="softmax"`: Use softmax for multi-class classification when labels are one-hot encoded (categorical).
#   If your `label_mode` was 'int' (integer labels), you might use `softmax` as well,
#   but it's crucial for categorical labels.
model.add(Dense(2, activation="softmax"))

# Optional: Print a summary of the model to see its architecture and number of parameters
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 220, 220, 32)        │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 110, 110, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 110, 110, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 108, 108, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 106, 106, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 53, 53, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 53, 53, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 179776)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │      92,045,824 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │           1,026 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 92,112,418 (351.38 MB)

 Trainable params: 92,112,418 (351.38 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

In [19]:
epochs = 40 
history = model.fit(train_ds,
                    epochs=epochs,
                    steps_per_epoch=len(train_ds),
                    validation_data=val_ds,
                    validation_steps=len(val_ds))

Epoch 1/40
3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 0.4340 - loss: 418.3614 - val_accuracy: 0.5000 - val_loss: 93.2605
Epoch 2/40
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.4961 - loss: 147.3366 - val_accuracy: 0.8500 - val_loss: 5.3309
Epoch 3/40
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.6693 - loss: 11.4847 - val_accuracy: 0.5500 - val_loss: 0.8228
Epoch 4/40
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.8012 - loss: 0.8389 - val_accuracy: 0.7000 - val_loss: 0.6801
Epoch 5/40
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.6693 - loss: 0.6200 - val_accuracy: 0.6500 - val_loss: 0.6859
Epoch 6/40
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.7795 - loss: 0.6564 - val_accuracy: 0.5500 - val_loss: 0.6885
Epoch 7/40
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.7127 - loss: 0.6697 - val_accuracy: 0.5500 - val_loss: 0.6896
Epoch 8/40
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.7088 - loss: 0.6728 - val_accuracy: 0.5500 - val_loss: 0.6904
Epoch 9/40

In [20]:
model.evaluate(train_ds)

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 165ms/step - accuracy: 1.0000 - loss: 0.0065


[0.009252588264644146, 1.0]

In [21]:
model.evaluate(val_ds)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step - accuracy: 0.5500 - loss: 5.7919


[5.79194450378418, 0.550000011920929]

In [ ]:
# so i have enter overfitting station 